# Manual Implementation of Problem 2 (Regression)

## Setting up the data

### Importing Libraries

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Loading & Splitting the dataset

In [30]:
df = pd.read_csv("California_Houses.csv")

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(df.isna().sum())

train_data_size = int(0.7 * len(df))
validation_data_size = int(0.15 * len(df))
test_data_size = len(df) - train_data_size - validation_data_size

train_data = df[:train_data_size]
validation_data = df[train_data_size:train_data_size + validation_data_size]
test_data = df[train_data_size + validation_data_size:]

Median_House_Value          0
Median_Income               0
Median_Age                  0
Tot_Rooms                   0
Tot_Bedrooms                0
Population                  0
Households                  0
Latitude                    0
Longitude                   0
Distance_to_coast           0
Distance_to_LA              0
Distance_to_SanDiego        0
Distance_to_SanJose         0
Distance_to_SanFrancisco    0
dtype: int64


### Extracting features matrix & Target Vector

In [31]:
x = train_data.iloc[:, 1:13]
t = train_data.iloc[:, 0]

X_vectors = {}
for col in x.columns:
    X_vectors[col] = x[col].values.reshape(-1, 1)

X = np.hstack(list(X_vectors.values()))
T = t.values.flatten()


X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X_scaled = (X - X_mean) / X_std

n_samples = X.shape[0]
X_biased = np.hstack((np.ones((n_samples, 1)), X_scaled))


## Direct solution of caclulating W*

$w^* = (X^{\top} X)^{-1} (X^{\top} T)$.

In [32]:
left_term = np.linalg.inv(X_biased.T @ X_biased)
right_term = X_biased.T @ T
W_star = left_term @ right_term
print("Learned Weights:", W_star)

Learned Weights: [207183.64947739  73339.23718052  11390.78283316  -9813.07726714
  39003.27192159 -50396.58892998  25030.66297081 -91070.56192616
 -58463.20851428 -12788.39669391 -34720.70712485  68437.02098179
   7981.2671976 ]


### Error report of closed form solution

In [33]:
y_closed_form = X_biased @ W_star
mse_closed_form = np.mean((T - y_closed_form) ** 2)
print("Closed Form MSE on Training Data:", mse_closed_form)

mae_closed_form = np.mean(np.abs(T - y_closed_form))
print("Closed Form MAE on Training Data:", mae_closed_form)

Closed Form MSE on Training Data: 4660061242.647201
Closed Form MAE on Training Data: 49736.20296263998


## Gradient Descent

In [34]:
w_gradient = np.zeros(X.shape[1])
b = 0.0
alpha = 0.01
N = len(T)
max_iter = 100000
i = 0

while True:
    y = np.dot(X_scaled, w_gradient) + b
    dw = (1/N) * np.dot(X_scaled.T, (y - T))
    db = (1/N) * np.sum(y - T)

    temp_w = w_gradient - alpha * dw
    temp_b = b - alpha * db

    if (np.sum(np.abs(temp_w - w_gradient)) < 1e-3 and abs(temp_b - b) < 1e-3):
        break

    
    w_gradient = temp_w
    b = temp_b

print("Learned Weights using Gradient Descent:", np.hstack((b, w_gradient)))

Learned Weights using Gradient Descent: [207183.64947739  73339.26283801  11390.84621904  -9813.26425025
  39004.27702662 -50396.39066188  25029.65362835 -91055.64221427
 -58467.87257439 -12788.68906468 -34718.63853906  68415.11508797
   7980.48882297]


### Error report of gradient descent

In [35]:
y_gradient = np.dot(X_scaled, w_gradient) + b
mse_gradient = np.mean((T - y_gradient) ** 2)
print("Gradient Descent MSE on Training Data:", mse_gradient)
mae_gradient = np.mean(np.abs(T - y_gradient))
print("Gradient Descent MAE on Training Data:", mae_gradient)

Gradient Descent MSE on Training Data: 4660061244.199259
Gradient Descent MAE on Training Data: 49736.23549778592
